In [1]:
import pandas as pd
import numpy as np

In [2]:
eth_merged_data_of_lyra_trades_and_deribit_ohlcv = pd.read_csv('./deribit lyra comparison/eth.csv')
btc_merged_data_of_lyra_trades_and_deribit_ohlcv = pd.read_csv('./deribit lyra comparison/btc.csv')

In [3]:
def create_put_call_parity(dataset, time_delta_in_minutes):
    dataset.reset_index(drop=True, inplace=True)
    # Assuming 'black_scholes' is your DataFrame
    # First, ensure datetime is in the proper pandas datetime format
    dataset['datetime'] = pd.to_datetime(dataset['datetime'], format='%Y-%m-%d %H:%M:%S')

    # Filter calls and puts into separate DataFrames
    calls = dataset[dataset['type'] == 'call'].copy()
    puts = dataset[dataset['type'] == 'put'].copy()
    # We ned to ensure data is sorted by datetime for the merge_asof to work effectively
    calls = calls.sort_values(by='datetime')
    puts = puts.sort_values(by='datetime')

    # Use merge_asof to find matching entries within a 5-minute window
    # merge_asof is used here because it's designed for time series data where exact matches are not necessary
    # 'direction': 'nearest' matches to the nearest key within the limit (5 min here)
    merged = pd.merge_asof(calls, puts, on='datetime', by=['strike', 'expiry'],
                        suffixes=('_call', '_put'),
                        tolerance=pd.Timedelta(f'{time_delta_in_minutes} minutes'),
                        direction='nearest')

    # Filter out pairs where calls and puts are not from the same time frame within the 5-minute limit
    merged = merged.dropna(subset=['type_put'])  # This assumes that the merge resulted in some NaNs for non-matching rows

    # Reset index if necessary
    merged.reset_index(drop=True, inplace=True)
    # Display the merged DataFrame
    return merged

put_call_parity_eth = create_put_call_parity(eth_merged_data_of_lyra_trades_and_deribit_ohlcv, 30)
put_call_parity_btc = create_put_call_parity(btc_merged_data_of_lyra_trades_and_deribit_ohlcv, 60)

In [4]:
# Calculate discrepancies based on put-call parityHJe
# Put-call parity formula: C - P = S - K * e^(-rT)
# For simplicity, assume 'S' (spot price) and 'r' (risk-free rate) are constant across the dataset

def calculate_put_call_discrepancies(dataset):
    # merged['interest_rate_call'] = merged['interest_rate_call'] / 100
    # Calculate the theoretical parity and the actual discrepancy
    dataset['theoretical_parity'] = dataset['selected_spot_price_call'] - dataset['strike'] * np.exp(-dataset['interest_rate_call'] * dataset['time_to_maturity_call'] / 100)
    dataset['observed_difference'] = dataset['lyra_price_call'] - dataset['lyra_price_put']
    dataset['discrepancy'] = dataset['observed_difference'] - dataset['theoretical_parity']
    # merged['selected_spot_price_call'].iloc[0] - merged['strike'].iloc[0] 
    # Filter the results to find significant discrepancies
    significant_discrepancies = dataset[abs(dataset['discrepancy']) > 0.01]  # You can adjust the threshold

    significant_discrepancies = significant_discrepancies[['datetime', 'expiry', 'strike', 'observed_difference', 'theoretical_parity', 'discrepancy', 'selected_spot_price_call', 'interest_rate_call', 'time_to_maturity_call', 'lyra_price_call', 'lyra_price_put']]
    significant_discrepancies.columns = ['trade_time', 'expiry', 'strike', 'C-P', 'S-Ke^-rT', 'discrepancy', 'S', 'r', 'T', 'C', 'S']
    return significant_discrepancies


put_call_parity_eth = calculate_put_call_discrepancies(put_call_parity_eth)
put_call_parity_btc = calculate_put_call_discrepancies(put_call_parity_btc)

In [5]:
put_call_parity_eth.to_csv('./put call parity/put_call_parity_eth.csv')
put_call_parity_btc.to_csv('./put call parity/put_call_parity_btc.csv')

In [6]:
put_call_parity_eth


,trade_time,expiry,strike,C-P,S-Ke^-rT,discrepancy,S,r,T,C,S
0,2024-06-07 17:06:01,2024-07-26 08:00:00,3700.0,140.90,-4.405906,145.305906,3678.3,3.5195,0.133117,409.9,269.00
1,2024-06-11 10:13:57,2024-07-26 08:00:00,4000.0,-266.27,-315.903996,49.633996,3667.0,3.4837,0.122949,170.6,436.87
2,2024-06-11 10:13:57,2024-07-26 08:00:00,4000.0,-266.27,-315.903996,49.633996,3667.0,3.4837,0.122949,170.6,436.87
3,2024-06-11 10:13:57,2024-07-26 08:00:00,4000.0,-266.27,-315.903996,49.633996,3667.0,3.4837,0.122949,170.6,436.87
4,2024-06-11 10:13:57,2024-07-26 08:00:00,4000.0,-266.27,-315.903996,49.633996,3667.0,3.4837,0.122949,170.6,436.87
...,...,...,...,...,...,...,...,...,...,...,...
839,2024-08-05 14:14:23,2024-08-30 08:00:00,3200.0,-877.20,-774.088441,-103.111559,2418.0,3.6546,0.067734,22.8,900.00
840,2024-08-05 14:14:23,2024-08-30 08:00:00,3200.0,-877.20,-774.088441,-103.111559,2418.0,3.6546,0.067734,22.8,900.00
841,2024-08-05 14:14:23,2024-08-30 08:00:00,3200.0,-877.20,-774.088441,-103.111559,2418.0,3.6546,0.067734,22.8,900.00
842,2024-08-05 14:14:23,2024-08-30 08:00:00,3200.0,-877.20,-774.088441,-103.111559,2418.0,3.6546,0.067734,22.8,900.00
